# Descrição

Um algorítmo que é capaz de trocar mensagens, de forma natural, com clientes, simulando o atendimento de um atendente humano.

Para conseguir simular naturalidade, vamos combinar diferentes técnicas para análise da mensagem enviada pelo cliente e posterior elaboração de uma resposta. 

As técnicas utilizadas são:

- modelagem de tópicos - para direcionar a resposta de acordo com o assunto detetado. Por exemplo, se o assunto (tópico) da mensagem for uma reclamação, construir a mensagem considerando esta informação. Se a mensagem for do tipo (tópico ) saudação, responder com outra saudação. 
- detecção de palavras-chave, como números de pedido ou algum tipo especial de produto no qual o cliente esteja interessado


Modelagem de tópicos é uma ferramenta de processamento de linguagem natural, concretizada através de aprendizado de máquiva não-supervisionado. Maior detalhamento será dado abaixo.

Para obter uma classificação das mensagens em tópicos, será necessário um pré-processamento das mesmas. 
É um processo extenso, descrito a seguir.


# Obter as mensagens através do BD

À primeira vista, a impressão que eu tenho é que os dados fornecidos para o desenvolvimento deste exercício foram retirados de um banco de dados MongoDB.

Para obter a informação na totalidade através de tal tipo de banco de dados, o seguinte query poderia ser realizado através do Mongo Shell ou Mongo Compass:

    db.mensagens.find(
        {
        $and: [
                {"user": {$in: ["user_1", "user_2", "attd_1", "sell_1"]}},
                {"message": {$ne: null}}
              ]
        }
    ).pretty()



Caso os dados estivessem armazenados no formato xml, poderíamos utilizar a ferramenta BeautifulSoup para obter as mensagens

    db = BeautifulSoup(open('bd.xml').read(), "lxml")
    mensagens = db.findAll('message') 

Caso os dados estejam gravados em texto puro, podemos fazer um scrapping da seguinte forma (versão simplificada):
    
    mensagens = []

    with open(bd.txt, "rb") as entrada:
            for linha in entrada:
                if linha.startswith('user'):
                    mensagens.append(linha)

## Obtendo mensagens através dos dados fornecidos

In [1]:
conversations = [

    # small talk
    [
        {'user': 'user_1', 'message': 'Oi, como você tá?', 'status': ''},
        {'user': 'user_2', 'message': 'tudo certinho e vc?   ', 'status': ''},
        {'user': 'user_1', 'message': ' Eu to bemm!!!', 'status': ''},
        {'user': 'user_1', 'message': 'tem alguma promoção   hoje?', 'status': ''},
        {'user': 'user_2', 'message': 'hj? temos umatv 50" \n ', 'status': ''},
    ],
    
    # customer service
    [
        {'user': 'user_1', 'message': 'Cadê o iphone?!'                              , 'status': 'payment_approved'},
        {'user': 'attd_1', 'message': 'Olá, o seu pagamento já foi aprovado'         , 'status': 'payment_approved'},
        {'user': 'attd_1', 'message': 'Quer dizer que o seu produto está a caminho'  , 'status': 'payment_approved'},
        {'user': 'user_1', 'message': 'Mas já faz cinco dias'                        , 'status': 'payment_approved'},
        {'user': 'attd_1', 'message': 'A nossa política de entrega são 5 dias úteis' , 'status': 'payment_approved'},
        {'user': 'user_1', 'message': 'Ahh é verdade xD'                             , 'status': 'payment_approved'},
    ],
    
    # sale
    [
        {'user': 'user_1', 'message': 'cade o iphone 10?'                            , 'status': 'shopping'},
        {'user': 'sell_1', 'message': 'Oi, o Iphone X está fora de estoque'          , 'status': 'shopping'},
        {'user': 'user_1', 'message': 'Huum, o que vcs tem disponível'               , 'status': 'shoppinng'},
        {'user': 'sell_1', 'message': 'Olha, temos o iphone X plus e o samsung s8'   , 'status': 'shopping'},
        {'user': 'user_1', 'message': 'O samsung é melhor que o iphone?'             , 'status': 'shopping'},
        {'user': 'sell_1', 'message': 'Eles são diferentes, mas são os melhores'     , 'status': 'shopping'},
    ],
]

In [2]:
# vamos obter as mensagens dos usuários apenas (user_1)
mensagens_user = []

for conversa in conversations:
    mensagens_user.append([mensagem["message"] for mensagem in conversa if mensagem["user"]=="user_1"])

mensagens_user = sum(mensagens_user, [])
mensagens_user

['Oi, como você tá?',
 ' Eu to bemm!!!',
 'tem alguma promoção   hoje?',
 'Cadê o iphone?!',
 'Mas já faz cinco dias',
 'Ahh é verdade xD',
 'cade o iphone 10?',
 'Huum, o que vcs tem disponível',
 'O samsung é melhor que o iphone?']

In [3]:
# vamos aumentar um pouquinho esta lista

adicionais = ["Bom dia!",
              "boa tarde!",
              "boa noite!",
              "eu gostaria de fazer uma reclamação",
              "eu gostaria de fazer uma troca",
              "eu gostaria de fazer uma devolução",
              "qual é o melhor celular?",
              "meu celular quebrou!",
              "não consigo fechar minha compra",
              "o site não funciona!",
              "quando vai ser entregue?",
              "quando vai chegar?",
              "qual é o prazo de entrega?",
              "o aparelho é muito ruim!",
              "boa tarde!",
              "boa noite!",
              "preciso de ajuda para escolher meu novo celular",
              "vocês aceitam cartão?",
              "a tv chegou com defeito",
              "o aparelho chegou com defeito",
              "o aparelho não funciona"
             ]

In [4]:
todas_msg = mensagens_user+adicionais
todas_msg

['Oi, como você tá?',
 ' Eu to bemm!!!',
 'tem alguma promoção   hoje?',
 'Cadê o iphone?!',
 'Mas já faz cinco dias',
 'Ahh é verdade xD',
 'cade o iphone 10?',
 'Huum, o que vcs tem disponível',
 'O samsung é melhor que o iphone?',
 'Bom dia!',
 'boa tarde!',
 'boa noite!',
 'eu gostaria de fazer uma reclamação',
 'eu gostaria de fazer uma troca',
 'eu gostaria de fazer uma devolução',
 'qual é o melhor celular?',
 'meu celular quebrou!',
 'não consigo fechar minha compra',
 'o site não funciona!',
 'quando vai ser entregue?',
 'quando vai chegar?',
 'qual é o prazo de entrega?',
 'o aparelho é muito ruim!',
 'boa tarde!',
 'boa noite!',
 'preciso de ajuda para escolher meu novo celular',
 'vocês aceitam cartão?',
 'a tv chegou com defeito',
 'o aparelho chegou com defeito',
 'o aparelho não funciona']

# Pré-processamento das mensagens


A modelagem de tópicos funciona a partir da análize BOW: bag of words.

Desta forma, precisamos transformar cada mensagem em uma "sacola de palavras".

Para isto, vamos processar as mensagens com um tokenizador que irá:

- transformar todas as palavras em minísculas
- remover números - não serão importantes para modelagem de tópicos
- remover palavras com menos que 2 caracteres (a princípio vamos ignorar expressões como XD ou =D)
- corrigir gramática
- obter apenas a "raiz" da palavra (stemmer)

Estas restrições são empregadas a fim de evitar agregar complexidade ao modelo quando não há ganho evidente.
Assim, consideramos que são equivalentes:
- celular|Celular|celulares
- menino|meninos|menina|meninas|mennino
- samsung|sansumg|Sannsungui

## Corretor gramatical

O corretor utilizado, projeto enchant, depende do dicionário local do sistema, então é necessário instalar o myspell:

    # sudo apt-get install myspell-pt-br

Além disso, vamos adicinar algumas palabras para o dicionário, como por ex. marcas e produtos:

In [5]:
termos = ["samsung", "motorola", "apple", "iphone", "pixel", "google"] # podem ser obtidos através da lista de produtos comercializados

arquivo = "/home/erickfis/.config/enchant/pt_BR.dic"
with open(arquivo, "w") as text_file:
    for termo in termos:
        print(termo, file=text_file)


In [6]:
import enchant
d = enchant.Dict('pt_BR')
d.check("samsung") # indica que agora o corretor reconhece a marca samsung

True

### Melhorando as sugestões do enchant

o método enchant.suggest() fornece uma lista de possíveis candidatos para correção. Geralmente a primeira opção é a correta, mas nem sempre será o caso:

In [7]:
d.suggest("samjung")

['samanga',
 'samonga',
 'samangã',
 'sambango',
 'junguetas',
 'junguismo',
 'sambongo',
 'samanguaiá',
 'sanamunda',
 'sangueja',
 'sununga',
 'samanguice',
 'samsung']

Por isso, devemos escolher a correção apropriada através de uma comparação de similaridade.

Para tanto, iremos utilizar os métodos 

    difflib.SequenceMatcher()
    difflib.SequenceMatcher().ratio()

O método sequenceMatcher compara pares de sequências de uma forma "human friendly".
O método ratio() mede a similaridade do par comparado. Valores acima de 0.6 indicam um match.

In [8]:
import difflib
difflib.SequenceMatcher(None, "samsung", "sony").ratio()

0.36363636363636365

In [9]:
difflib.SequenceMatcher(None, "samsung", "sansumg").ratio()

0.7142857142857143

In [10]:
def corretor(palavra):
    
    melhor_sugestao = ""
    melhor_ratio = 0 # começando com similaridade 0

    sugestoes = set(d.suggest(palavra))
    for sugestao in sugestoes:
        tmp = difflib.SequenceMatcher(None, palavra, sugestao).ratio()
        if tmp > melhor_ratio:
            melhor_sugestao = sugestao
            melhor_ratio = tmp # aumenta o nível para próximas comparações

    return melhor_sugestao


In [11]:
corretor("samsungui")

'samsung'

In [12]:
corretor("samjung")

'samsung'

## O tokenizador

O tokenizador irá transformar, efetivamente, as mensagens em "sacolas de palavras".

Ele deve:
- aplicar lowercase
- separar a mensagem em tokens, removendo números
- remover palavras que contenham menos de 2 letras
- aplicar corretor gramatical
- reduzir palavras para a forma mais básica

In [13]:
import nltk
from nltk.stem import snowball
from nltk.corpus import stopwords

stopWords = set(stopwords.words('portuguese'))
ptlem = snowball.PortugueseStemmer()

import re # para expressões regulares com o método findall()

In [14]:
def tokenizador(mensagem, tamanho=2, corrige=1, stem=1):

    mensagem = mensagem.lower() # lowercase, pois consideramos "Celular" == "celular"
    mensagem = mensagem.replace("boa", "bom") # boa tarde == bom dia == saudação   
    
    # obter tokens, inclusive acentuados, eliminando espaço e pontuação
    tokens = re.findall("[-'a-zA-ZÀ-ÖØ-öø-ÿ]+", mensagem) 
    
    ## remove palavras com tamanho <= 2, ex XD
    tokens = [token for token in tokens if len(token) > tamanho] 
    
    # se as palavras estiverem erradas, trocar por sugestão do enchant, senão, usar ela mesma
    if corrige:
        tokens = [corretor(token) if not d.check(token) else token for token in tokens]
    
    # stemmer para obter somente a "raiz" das palavras, Ex: "menino" == "menina" == "menin"
    if stem:
        tokens = [ptlem.stem(t) for t in tokens] 
    
    # não vamos remover stop words, porque as sentenças já são pequenas demais
    #tokens = [t for t in tokens if t not in stopWords] # remove stopwords
    
    return tokens

### Aplicando o tokenizador

Vamos agora transformar as mensagens do nosso DB em tokens para posteriormente obter nossa matriz de frequência para termos-documentos:

In [15]:
tokens = map(lambda x: tokenizador(x), todas_msg)
msg_pro = list(tokens)
msg_pro # mensagens processadas

[['com', 'voc'],
 ['bem'],
 ['tem', 'algum', 'promoçã', 'hoj'],
 ['cad', 'iphon'],
 ['mas', 'faz', 'cinc', 'dias'],
 ['ah', 'verdad'],
 ['cad', 'iphon'],
 ['hum', 'que', 'vs', 'tem', 'dispon'],
 ['samsung', 'melhor', 'que', 'iphon'],
 ['bom', 'dia'],
 ['bom', 'tard'],
 ['bom', 'noit'],
 ['gost', 'faz', 'uma', 'reclam'],
 ['gost', 'faz', 'uma', 'troc'],
 ['gost', 'faz', 'uma', 'devolu'],
 ['qual', 'melhor', 'celul'],
 ['meu', 'celul', 'quebr'],
 ['nã', 'consig', 'fech', 'minh', 'compr'],
 ['sit', 'nã', 'funcion'],
 ['quand', 'vai', 'ser', 'entreg'],
 ['quand', 'vai', 'cheg'],
 ['qual', 'praz', 'entreg'],
 ['aparelh', 'muit', 'ruim'],
 ['bom', 'tard'],
 ['bom', 'noit'],
 ['precis', 'ajud', 'par', 'escolh', 'meu', 'nov', 'celul'],
 ['vocês', 'aceit', 'cartã'],
 ['cheg', 'com', 'defeit'],
 ['aparelh', 'cheg', 'com', 'defeit'],
 ['aparelh', 'nã', 'funcion']]

# Modelagem de tópicos

Este é o ponto central da solução aqui apresentada: o algorítmo deve ser capaz de compreender o assunto (tópico) da mensagem enviada pelo cliente para então começar a montar uma resposta direcionada.

A fim de melhor aproveitar o banco de dados existente (presumo que sejam dezenas de milhares de registros), vamos obter os tópicos através de aprendizado de máquina não-supervisionado, ou seja, não vamos fornecer ao algorítmo, previamente, uma set de treino com os rótulos já determinados.

Do contrário, seria necessária intervenção humana para ler as mensagens, uma a uma, e atribuir pessoalmente um rótulo a elas. 
Esse processo não é viável para o nosso caso (presumo que sejam dezenas de milhares de registros...). 
Além disso, a tecnologia nos permite abordar o problema de uma forma muito mais eficiente.




## Método LDA - Latent Dirichlet Allocation


Para natural language processing, um aprendizado de máquina não supervisionado pode ser obtido através de algumas técnicas diferentes, entre elas:
- redução de dimensionalidade PCA ou SVD
- Latent semantic analysis - LSA
- LDA - Latent Dirichlet Allocation

O LDA é um caso especial de análise probabilística de semântica latente onde a distribuição "prior" de tópicos é considerada como sendo do tipo beta multivariável, ou seja, uma distribuição Dirichlet. 

O LDA tem como principal vantagem a capacidade de encontrar tópicos intermediários entre os tópicos que seriam sugeridos pelo método LSA/PCA/SVD (que utilizam redução de dimensionalidade, trabalha com os componentes principais e, portanto, sugerem tópicos ortogonais), evitando assim overfitting e aumentando a acurácia.

Por outro lado, exige mais tempo de processamento que o demais.

O pacote *gensim* traz as ferramentas necessárias para implementar uma análise LDA em python.

In [16]:
from gensim import corpora, models

dicionario = corpora.Dictionary(msg_pro) # obtendo um dicionário através do nosso DB

/data/data-erick/anaconda3/lib/python3.6/site-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.22) or chardet (2.3.0) doesn't match a supported version!
  RequestsDependencyWarning)


In [17]:
corpo = [dicionario.doc2bow(msg) for msg in msg_pro] # matriz de termos para cada documento

## Treinando nosso modelo LDA

Vamos começar estimando 5 tópicos, com 100 passadas sobre nosso DB. Vamos também cronometrar o tempo gasto no processo.

O parâmetro alpha diz respeito à densidade documento-tópico. Valores altos indicam que cada documento contém mais tópicos. Como esperamos que a mensagem do cliente contenha apenas 1 tópico, vamos usar alpha = .2

O número de passes indica a quantidade de iterações que serão realizadas para obter convergência dos resultados.

In [18]:
import time
import random

start = time.time()
random.seed(95276)
modelo = models.ldamodel.LdaModel(corpo, gamma_threshold=.01, minimum_phi_value=.005, per_word_topics=True, minimum_probability=.01,
    num_topics=6, id2word = dicionario, passes=100, alpha=.2, eta=10, random_state=95276)

    # alpha (relação documento/tópico) e eta (relação tópicos-palavras) 
    # poderiam ser parametrizados para "aprender com os dados" - opção "auto"
    
print("\n --- %s segundos ---" % round((time.time() - start),4))



 --- 2.8609 segundos ---


### Visualizando os 3 principais termos de cada tópico

In [19]:
modelo.print_topics(num_topics=6, num_words=3)


[(0, '0.021*"cheg" + 0.021*"com" + 0.020*"aparelh"'),
 (1, '0.023*"faz" + 0.021*"gost" + 0.021*"uma"'),
 (2, '0.020*"tem" + 0.018*"hoj" + 0.018*"promoçã"'),
 (3, '0.025*"bom" + 0.020*"tard" + 0.020*"noit"'),
 (4, '0.021*"celul" + 0.021*"iphon" + 0.020*"meu"'),
 (5, '0.021*"nã" + 0.019*"funcion" + 0.018*"minh"')]

### Serializando o modelo

Para não ter que treinar novamente (salvando o modelo para o disco)

In [20]:
import pickle

# grava no disco
pickle.dump(modelo, open("modelo_lda.model", 'wb'))
pickle.dump(dicionario, open("dicionario.model", 'wb'))

# carregando de volta
modelo_salvo = pickle.load(open("modelo_lda.model", 'rb'))
dicionario = pickle.load(open("dicionario.model", 'rb'))

modelo_salvo.print_topics(num_topics=6, num_words=3)


[(0, '0.021*"cheg" + 0.021*"com" + 0.020*"aparelh"'),
 (1, '0.023*"faz" + 0.021*"gost" + 0.021*"uma"'),
 (2, '0.020*"tem" + 0.018*"hoj" + 0.018*"promoçã"'),
 (3, '0.025*"bom" + 0.020*"tard" + 0.020*"noit"'),
 (4, '0.021*"celul" + 0.021*"iphon" + 0.020*"meu"'),
 (5, '0.021*"nã" + 0.019*"funcion" + 0.018*"minh"')]

## Análise visual  dos tópicos obtidos

Quanto maior a distinção entre os agrupamentos identificados, melhor.
Para obter tal distinção, devem ser ajustados os parâmetros da preparação do modelo lda.


In [21]:
import pyLDAvis.gensim
vis = pyLDAvis.gensim.prepare(modelo_salvo, corpo, dicionario)
pyLDAvis.enable_notebook()

vis

/data/data-erick/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      23.377256        1       1 -0.000287  0.000732
1      18.865831        1       2 -0.000522 -0.000653
3      17.556702        1       3 -0.000071  0.000054
0      16.841709        1       4  0.000850 -0.000148
5      14.169906        1       5  0.000045 -0.000018
2       9.188601        1       6 -0.000016  0.000032, topic_info=     Category      Freq     Term     Total  loglift  logprob
term                                                        
6     Default  1.000000      tem  1.000000  30.0000  30.0000
38    Default  1.000000       nã  1.000000  29.0000  29.0000
21    Default  1.000000      bom  1.000000  28.0000  28.0000
3     Default  1.000000    algum  1.000000  27.0000  27.0000
4     Default  1.000000      hoj  1.000000  26.0000  26.0000
5     Default  1.000000  promoçã  1.000000  25.0000  25.0000
18    Default  1.000000       vs  1.000000  24.0000  24.0000
16    Default  1.000000      hum  1.000000  23.0000  23.0000
15    Default  1.000000   dispon  1.000000  22.0000  22.0000
39    Default  1.000000  funcion  1.000000  21.0000  21.0000
17    Default  1.000000      que  1.000000  20.0000  20.0000
45    Default  1.000000     cheg  1.000000  19.0000  19.0000
0     Default  1.000000      com  1.000000  18.0000  18.0000
47    Default  1.000000  aparelh  1.000000  17.0000  17.0000
56    Default  1.000000    cartã  1.000000  16.0000  16.0000
57    Default  1.000000    vocês  1.000000  15.0000  15.0000
55    Default  1.000000    aceit  1.000000  14.0000  14.0000
37    Default  1.000000     minh  1.000000  13.0000  13.0000
36    Default  1.000000     fech  1.000000  12.0000  12.0000
35    Default  1.000000   consig  1.000000  11.0000  11.0000
34    Default  1.000000    compr  1.000000  10.0000  10.0000
40    Default  1.000000      sit  1.000000   9.0000   9.0000
14    Default  1.000000   verdad  1.000000   8.0000   8.0000
13    Default  1.000000       ah  1.000000   7.0000   7.0000
2     Default  1.000000      bem  1.000000   6.0000   6.0000
41    Default  1.000000   entreg  1.000000   5.0000   5.0000
58    Default  1.000000   defeit  1.000000   4.0000   4.0000
44    Default  1.000000      vai  1.000000   3.0000   3.0000
42    Default  1.000000    quand  1.000000   2.0000   2.0000
11    Default  1.000000      faz  1.000000   1.0000   1.0000
...       ...       ...      ...       ...      ...      ...
46     Topic6  0.147120     praz  1.591100   0.0063  -4.0832
40     Topic6  0.145674      sit  1.588519  -0.0020  -4.0931
34     Topic6  0.145327    compr  1.588513  -0.0044  -4.0954
35     Topic6  0.145327   consig  1.588513  -0.0044  -4.0954
36     Topic6  0.145327     fech  1.588513  -0.0044  -4.0954
37     Topic6  0.145327     minh  1.588513  -0.0044  -4.0954
1      Topic6  0.145336      voc  1.592250  -0.0066  -4.0954
48     Topic6  0.145321     muit  1.592267  -0.0068  -4.0955
49     Topic6  0.145321     ruim  1.592267  -0.0068  -4.0955
43     Topic6  0.145311      ser  1.592251  -0.0068  -4.0955
22     Topic6  0.145335      dia  1.593832  -0.0077  -4.0954
9      Topic6  0.145311     cinc  1.595615  -0.0089  -4.0955
10     Topic6  0.145311     dias  1.595615  -0.0089  -4.0955
12     Topic6  0.145311      mas  1.595615  -0.0089  -4.0955
28     Topic6  0.145311     troc  1.595616  -0.0089  -4.0956
29     Topic6  0.145311   devolu  1.595616  -0.0089  -4.0956
41     Topic6  0.147138   entreg  1.617250  -0.0099  -4.0831
31     Topic6  0.147149     qual  1.627293  -0.0160  -4.0830
39     Topic6  0.145701  funcion  1.612353  -0.0167  -4.0929
38     Topic6  0.145733       nã  1.634753  -0.0303  -4.0927
24     Topic6  0.145372     noit  1.621570  -0.0246  -4.0951
23     Topic6  0.145371     tard  1.621570  -0.0247  -4.0951
21     Topic6  0.145453      bom  1.704802  -0.0741  -4.0946
7      Topic6  0.145375      cad  1.638375  -0.0349  -4.0951
8      Topic6  0.145386    iphon  1.674641  -0.0

## Demonstrando os tópicos identificados

In [23]:
def get_topico(mensagem):
    tokens = tokenizador(mensagem)
    mapa = dicionario.doc2bow(tokens)
    # modelo_salvo[mapa] irá retornar todos os tópicos possíveis e uma 
    # pontuação que informa a probabilidade de qua a mensagem pertençaàquele tópico
    
    # além de ordenar para obter o tópico com maior probabilidade, vamos estabelecer a seguinte regra:
    # se a probabilidade for menor que 60%, atribuir à categoria outros (número 6, no mapa que iremos criar)
    
    guess = sorted(modelo_salvo[mapa][0], key=lambda y: y[1], reverse=True)[0] # retorna o tópico mais provável e sua pontuação

    return 6 if guess[1] < .6 else guess[0]

In [24]:
import pandas as pd
df = pd.DataFrame({"mensagens": todas_msg})

df["tokens"] = df.mensagens.apply(lambda x: tokenizador(x))

df["topico"] = df.mensagens.apply(lambda x: get_topico(x))

df.sort_values(by="topico")

,mensagens,tokens,topico
0,"Oi, como você tá?","[com, voc]",0
27,a tv chegou com defeito,"[cheg, com, defeit]",0
20,quando vai chegar?,"[quand, vai, cheg]",0
19,quando vai ser entregue?,"[quand, vai, ser, entreg]",0
28,o aparelho chegou com defeito,"[aparelh, cheg, com, defeit]",0
13,eu gostaria de fazer uma troca,"[gost, faz, uma, troc]",1
12,eu gostaria de fazer uma reclamação,"[gost, faz, uma, reclam]",1
14,eu gostaria de fazer uma devolução,"[gost, faz, uma, devolu]",1
4,Mas já faz cinco dias,"[mas, faz, cinc, dias]",1
22,o aparelho é muito ruim!,"[aparelh, muit, ruim]",2


## Identificando os tópicos

Após aplicação do modelo para identificar os agrupamentos de mensagens,
devemos analizar manualmente uma pequena amostra de cada grupo para então rotulá-los de forma amigável.

Importante notar que, caso o modelo seja treinado novamente, mas com outros parâmetros, a ordem dos rótulos atribuídos a cada grupo pode mudar, o que pode arruinar todo o restante do trabalho.

Esta rotulação e verificação manual dos grupos é, portanto, de suma importância.

In [25]:
# vizualisando uma fração das mensagens de cada grupo, para ajudar a definir o título de cada tópico
# foi usado 80% porque nosso db é muuuuito pequeno

df.groupby('topico').apply(lambda x: x.sample(frac=.8))

mensagens  \
topico                                                       
0      28                    o aparelho chegou com defeito   
       20                               quando vai chegar?   
       0                                 Oi, como você tá?   
       27                          a tv chegou com defeito   
1      12              eu gostaria de fazer uma reclamação   
       13                   eu gostaria de fazer uma troca   
       14               eu gostaria de fazer uma devolução   
2      7                    Huum, o que vcs tem disponível   
       2                       tem alguma promoção   hoje?   
       22                         o aparelho é muito ruim!   
3      11                                       boa noite!   
       9                                          Bom dia!   
       10                                       boa tarde!   
       21                       qual é o prazo de entrega?   
       23                                       boa tarde!   
4      15                         qual é o melhor celular?   
       8                  O samsung é melhor que o iphone?   
       3                                   Cadê o iphone?!   
       25  preciso de ajuda para escolher meu novo celular   
       16                             meu celular quebrou!   
5      29                          o aparelho não funciona   
       5                                  Ahh é verdade xD   
       18                             o site não funciona!   
6      1                                     Eu to bemm!!!   

                                                 tokens  topico  
topico                                                           
0      28                  [aparelh, cheg, com, defeit]       0  
       20                            [quand, vai, cheg]       0  
       0                                     [com, voc]       0  
       27                           [cheg, com, defeit]       0  
1      12                      [gost, faz, uma, reclam]       1  
       13                        [gost, faz, uma, troc]       1  
       14                      [gost, faz, uma, devolu]       1  
2      7                    [hum, que, vs, tem, dispon]       2  
       2                     [tem, algum, promoçã, hoj]       2  
       22                         [aparelh, muit, ruim]       2  
3      11                                   [bom, noit]       3  
       9                                     [bom, dia]       3  
       10                                   [bom, tard]       3  
       21                          [qual, praz, entreg]       3  
       23                                   [bom, tard]       3  
4      15                         [qual, melhor, celul]       4  
       8                  [samsung, melhor, que, iphon]       4  
       3                                   [cad, iphon]       4  
       25  [precis, ajud, par, escolh, meu, nov, celul]       4  
       16                           [meu, celul, quebr]       4  
5      29                        [aparelh, nã, funcion]       5  
       5                                   [ah, verdad]       5  
       18                            [sit, nã, funcion]       5  
6      1                                          [bem]       6

In [26]:
# um dicionário de tópicos
rotulos = ["entrega", "solicitação", "indicação", "saudação", "comparacao", "problema", "outros"]
rotulos = dict(zip(range(7), rotulos))
rotulos

{0: 'entrega',
 1: 'solicitação',
 2: 'indicação',
 3: 'saudação',
 4: 'comparacao',
 5: 'problema',
 6: 'outros'}

In [27]:
df["rotulo"] = df.topico.replace(rotulos)
df

,mensagens,tokens,topico,rotulo
0,"Oi, como você tá?","[com, voc]",0,entrega
1,Eu to bemm!!!,[bem],6,outros
2,tem alguma promoção hoje?,"[tem, algum, promoçã, hoj]",2,indicação
3,Cadê o iphone?!,"[cad, iphon]",4,comparacao
4,Mas já faz cinco dias,"[mas, faz, cinc, dias]",1,solicitação
5,Ahh é verdade xD,"[ah, verdad]",5,problema
6,cade o iphone 10?,"[cad, iphon]",4,comparacao
7,"Huum, o que vcs tem disponível","[hum, que, vs, tem, dispon]",2,indicação
8,O samsung é melhor que o iphone?,"[samsung, melhor, que, iphon]",4,comparacao
9,Bom dia!,"[bom, dia]",3,saudação


## Aplicando o modelo para novas mensagens

In [28]:
mensagem = "qundo vai chegar?"
rotulos[get_topico(mensagem)]

'entrega'

In [29]:
mensagem = "ah!"
rotulos[get_topico(mensagem)]

'outros'

 # Detecção de palavras-chave
 
 As palavras-chave, associadas à informação obtida na modelagem de tópicos, ajudará a construir uma resposta mais útil.
 
 ## Número de pedido
 

In [30]:
def get_pedido(msg):
    tokens = re.findall("[0-9]+", msg)
    pedido = [token for token in tokens if len(token)==10] # o número de pedido contém 10 números
    
    return pedido
    
get_pedido("comprei 10 iphone, o pedido é 1234567890")

['1234567890']

## Tipo de aparelho (ou objeto de interesse do cliente)

In [31]:
def get_aparelho(msg):
    tokens = tokenizador(msg, tamanho=1, stem=0, corrige=0)
    
    # uma lista de objetos de interesse do cliente - alimentar com os produtos vendidos
    # podemos utilizar NLP POS - parts of speech para reconhecer o objeto, mas aqui acabou o tempo =D
    
    possibilidades = ["tv", "celular", "televisão", "microondas", "site", "geladeira", "pneu", "roupa"] 
    
    aparelho = [token for token in tokens if token in possibilidades]
    return aparelho
    

get_aparelho("preciso de indicação para tv e celular!")
#get_aparelho("preciso de indicação para tv")

['tv', 'celular']

# Preparando a resposta

A resposta ao usuário deverá ser composta de acordo com:

- o tópico escolhido - cada tópico terá uma função auxiliar
- palavras-chave detectadas

Serão utilizadas as funções auxiliares:
- get_topico() - ok
- get_aparelho() - ok
- get_pedido() - ok
- status_pedido()
- faz_cumprimento() 
- nao_entendeu()


In [32]:
import datetime

def faz_cumprimento(tipo="cumprimenta"):
    agora = datetime.datetime.now()
    hora = agora.hour
    
    if tipo == "cumprimenta":
        saudacao = "Boa noite"
        if hora in range(12,19):
            saudacao = "Boa tarde"
        if hora in range(6,13):
            saudacao = "Boa bom dia"
        
        mensagem = saudacao + ", caro cliente!"
            
    else:
        mensagem = "São {} horas".format(hora)
        
    return mensagem
        
faz_cumprimento()

'Boa tarde, caro cliente!'

In [33]:
faz_cumprimento("hora")

'São 15 horas'

In [34]:
def status_pedido(numero):
    # query sql ou afim para buscar no sistema informações sobre o andamento do pedido
    # %%sql
    # select pedido, andamento, data
    # from pedidos
    # where pedido = numero
    # order by data
    
    return(" Status do pedido {}, de acordo com o db: status".format(numero))

status_pedido(128312983)

' Status do pedido 128312983, de acordo com o db: status'

In [35]:
contador_erros = 0

In [36]:
def nao_entendeu():
    global contador_erros
    mensagens = ["está tudo bem aqui na Terra! Em que podemos ajudá-lo?",
                "Por favor seja mais específico...",
                "Eu sou apenas um robô cansado. Repita, devagar...",
                "isso é uma pegadinha?"
                ]
    
    if not contador_erros:
        mensagem = "{} {} e {}".format(faz_cumprimento(), faz_cumprimento("hora"), mensagens[0])
    else:
        mensagem = mensagens[contador_erros]
    
    contador_erros += 1
    if contador_erros == 4:
        contador_erros = 1
    
    return mensagem

nao_entendeu()


'Boa tarde, caro cliente! São 15 horas e está tudo bem aqui na Terra! Em que podemos ajudá-lo?'

In [37]:
nao_entendeu() # cada vez retorna uma resposta diferente

'Por favor seja mais específico...'

In [38]:
rotulos # para guiar a contrução das respostas

{0: 'entrega',
 1: 'solicitação',
 2: 'indicação',
 3: 'saudação',
 4: 'comparacao',
 5: 'problema',
 6: 'outros'}

In [ ]:
Vamos agora criar um dicionário contendo respostas para cada classe (tópico) de mensagem. 
A reposta 0 da classe assume que o cliente não informou palavras-chave. Por outro lado, a resposta 1 assume que sim:

In [39]:
m_entrega = ["Nosso prazo é de 5 dias úteis. Por favor informe o número do pedido que eu vou buscar mais detalhes",
             "Nosso prazo é de 5 dias úteis. Vamos checar o status do pedido {}."]

m_solicitacao = ["Pois não, vamos por partes. Qual é o número do pedido?",
                 "Ok. Vamos checar o status do pedido {}."]

m_indicacao = ["Você quer indicação para qual tipo de aparelho?",
               "Aqui estão as melhores ofertas para {}."]

m_comparacao = ["Nós podemos ajudar a escolher. Qual tipo de aparelho você está procurando?",
                "Estas são as melhores opções para {}."]

m_problema = ["Calma, tudo tem jeito! O que está acontecendo?",
              "Calma, vamos resolver esse problema com {} da melhor forma possível."]

todas_repostas = {
    0: m_entrega,
    1: m_solicitacao,
    2: m_indicacao,
    3: faz_cumprimento(),
    4: m_comparacao,
    5: m_problema,
    6: nao_entendeu()
}

todas_repostas[3]

'Boa tarde, caro cliente!'

## Respondendo ao cliente

Agora que temos todas as funções auxiliares definidas, podemos montar nossa resposta ao cliente:

In [40]:
def sub_resp(var, topico): # ok, mais uma auxiliar, para acelerar o "coding" da main
    if var:
        if var[0].isdigit():
            resposta = todas_repostas[topico][1].format(var[0]) + status_pedido(var[0])
        else:
            resposta = todas_repostas[topico][1].format(var[0])
    else:
        resposta = todas_repostas[topico][0]
            
    return resposta

def responde_cliente(mensagem): # agora sim

    topico = get_topico(mensagem)
    n_pedido = get_pedido(mensagem)
    aparelho = get_aparelho(mensagem)
    
    if topico in [0,1]:
        return sub_resp(n_pedido, topico)
    
    if topico in [2,4,5]:
        return sub_resp(aparelho, topico)
    
    if topico == 3:
        return todas_repostas[topico]
    else:
        return todas_repostas[topico]
    
contador_erros = 0

In [41]:
mensagem = "tem promoção de tv ou celular hoje?"

In [42]:
responde_cliente(mensagem)

'Aqui estão as melhores ofertas para tv.'

In [43]:
responde_cliente("bom dia!")

'Boa tarde, caro cliente!'

In [44]:
responde_cliente("whacka whacka whacka!")

'Eu sou apenas um robô cansado. Repita, devagar...'

In [45]:
responde_cliente("o site tá ruim")

'Calma, vamos resolver esse problema com site da melhor forma possível.'

In [46]:
responde_cliente("quando chega o pedido 1234567890")

'Nosso prazo é de 5 dias úteis. Vamos checar o status do pedido 1234567890. Status do pedido 1234567890, de acordo com o db: status'

# Escalabilidade

A fim de otimizar o tempo de computação, o pacote *gensim* fornece algumas formas alternativas para treinamento do modelo LDA.

Entre as opções estão: 

- montar um cluster de computadores e processar o trabalho neste cluster de forma distribuída;
- em vez de realizar 100 ou n "passadas" por todas as mensagens, no modo batch, podemos utilizar o modo "on line", que considere apenas um subconjunto das mensagens, de tamanho m, monta o modelo, processa mais um subconjunto, atualiza o modelo, processa mais um sobconjunto e assim por diante;
- os dois anteriores

A seguir, vamos comparar o modo online com o modo batch (10 passadas). Para isso, vamos simular um "corpo" com tamanho 1000 vezes maior que o nosso "corpo de mensagens"


In [47]:
import time
corpo2 = corpo*1000

In [48]:
start = time.time()

modelo_bath = models.ldamodel.LdaModel(corpo2, num_topics=6, id2word = dicionario, passes=10)
    
print("\n --- %s segundos ---" % round((time.time() - start),4))


 --- 75.273 segundos ---


Foram necessários 75 segundos para processar o modelo em modo batch nesta máquina (Core i5 com 4Gb de RAM)

In [49]:
start = time.time()

modelo_online = models.ldamodel.LdaModel(corpo2, num_topics=6, id2word = dicionario, update_every=1, chunksize=100, passes=1)
    
print("\n --- %s segundos ---" % round((time.time() - start),4))


 --- 6.0613 segundos ---


Foram necessários 7 segundos para processar o modelo em modo on line

In [50]:
modelo_bath.print_topics(num_words=3)

[(0, '0.091*"celul" + 0.091*"uma" + 0.091*"gost"'),
 (1, '0.091*"com" + 0.091*"defeit" + 0.091*"cheg"'),
 (2, '0.182*"nã" + 0.182*"funcion" + 0.091*"aparelh"'),
 (3, '0.182*"vai" + 0.182*"quand" + 0.182*"entreg"'),
 (4, '0.125*"iphon" + 0.125*"cad" + 0.063*"ruim"'),
 (5, '0.231*"bom" + 0.154*"noit" + 0.077*"meu"')]

In [51]:
modelo_online.print_topics(num_words=3)

[(0, '0.154*"qual" + 0.153*"melhor" + 0.077*"entreg"'),
 (1, '0.264*"bom" + 0.106*"tard" + 0.106*"noit"'),
 (2, '0.200*"tem" + 0.200*"que" + 0.200*"hum"'),
 (3, '0.210*"faz" + 0.158*"gost" + 0.158*"uma"'),
 (4, '0.100*"celul" + 0.100*"meu" + 0.099*"cad"'),
 (5, '0.158*"com" + 0.106*"aparelh" + 0.106*"cheg"')]

# Considerações finais

## Foco no problema, foco na solução

Eu sei que, inicialmente, a proposta era comparar através de uma forma de pontuação a resposta obtida pelo modelo com a resposta tipicamente fornecida por um atendente real. MAS, focar em imitar o atendente real pode não ser o mais indicado até mesmo porque nem sempre as conversas tomam rumos eficazes.


### Scoring

De qualquer forma, caso se queira mesmo comparar as repostas registradas no BD com as fornecidas pelo modelo, pode-se empregar o método difflib.SequenceMatcher().ratio() para scoring. Exemplos:


In [52]:
import difflib
difflib.SequenceMatcher(None, "resposta do atendente humano", "humano atendente do resposta").ratio()

0.5

In [53]:
difflib.SequenceMatcher(None, "resposta do atendente humano", "resposta do atendente").ratio()

0.8571428571428571

In [54]:
difflib.SequenceMatcher(None, "resposta do atendente humano", "outra resposta parecida com humano").ratio()

0.5806451612903226

## Outras possibilidades

NLP POS - Parts of speech poderia ter sido empregado para detectar palavras-chave, a fim de guiar a preparação da resposta ao cliente.

Além disso, utilizei uma solução extremamente simples para correção gramatical. Existem possibilidades muito mais sofisticadas mas que dependem de um treino realizado sobre um texto de referência, para "aprender" o vocabulário.

Eu desenvolvi este esboço de solução em Python a fim de demonstrar minhas habilidades nesta linguagem.
No entanto, a linguagem R, na qual sou ainda mais fluente, possui ferramentas mais ricas, tanto para análise de dados como para elaboração de relatórios, apresentações, dashboards e afins. Vide, por exemplo, o meu repositório no github, onde eu apresento relatórios, apresentações e dashboards sobre assuntos variados  - https://github.com/erickfis


## Sobre mim

Eu adoro criar e ser desafiado. 


Aqui vai um resumo das minhas habilidades:

    - R & Python (Pandas, SciPy, scikit-learn, dplyr, caret)
    - BI Tableau, Google Vis, ggPlot, Shiny Dashboards
    - MySQL / Teradata, NoSQL, MongoDB, Apache Cassandra
    - Hadoop, Amazon Web Services – AWS EC2
    - Machine Learning & Modelos de regresão linear, Árvores de Classificação e etc
    - Natural Language Processing – NLP
    - Clustering / Regras de associação
    - Inferência estatística, testes A/B
    - Análise exploratória
    - Git / Github
    - Rmarkdown Reproducible Research / Jupyter Notebooks
    - Fluente em inglês
    - Físico formado pela USP
    
*Erick Gomes Anastácio*
